# Victim

## SBOX lookup

Firmware under analysis is placed in [sbox.c](../src/sbox.c).

The file contains the basic steps of an AES SBOX lookup; 
i.e. step "AddKey" and "SubByte" of the first round of an AES 128 encryption.

Two classes of countermeasures are implemented:

- **Hiding**: 
  The order of loop execution can be randomized.
  Thus, the probability that one specific key byte is processed can be pushed down to 1/32.

  Note: While a genuine AES 128 has a key of 16 bytes this victim code extends the loop to 32 bytes.
  This gives more flexibility in hiding the actual operation not only in one out of 16 but in one out of 32.
- **Masking**:
  The SBOX can be masked with a one byte input and another one byte output mask.

All countermeasures can be controlled from outside by adjusting the respective "randomness".

The step of masking the SBOX is _not_ part of the analysis; i.e. this part is not captured.
Instead it is done in a separate function called `sbox_prepare`.

To perform 32 lookups the buffers for key, input, and working state are extended to 32 bytes. 
The last 16 bytes are filled with random values.

## Wrapper

While `sbox.c` contains only the genuine code performing the code under analysis, further code is required to perform capturing of side-channel traces.
This can be found in the folder [capture](../capture) and the respective target.


## Examples

1. **Plain lookup**: The plain lookup is performed by setting 

   - `rand->random_loop_order = 0`
   - `rand->sbox_mask_in = 0`
   - `rand->sbox_mask_out = 0`

   This virtually leads to the following code:
   
   ```c
    {
        _trigger_high();

        for (uint_fast8_t i = 0; i < 32; i++)
        {
            uint8_t lookup = aes_sbox[key[i] ^ extended_input[i]];
            working_state[i] = lookup;
        }

        _trigger_low();
    }
    ```

2. **Random loop order**: 
   When we set `rand->random_loop_order` to something else than `0`, we gain virtually the following code:
   
   ```c
    {
        _trigger_high();

        uint_fast8_t random_loop_order = rand->random_loop_order & 0x1F;
        for (uint_fast8_t i = 0; i < 32; i++)
        {
            uint8_t lookup = aes_sbox[key[i ^ random_loop_order] ^ extended_input[i ^ random_loop_order]];
            working_state[i ^ random_loop_order] = lookup;
        }

        _trigger_low();
    }
    ```

   We realize that there are multiple "security levels" that can be achieved:

   - **1 bit randomness**: `random_loop_order = [0..1]`: <br/>
     If `random_loop_order` is either `0` or `1` lookup of the first key byte is performed either for `i = 0` or for `i = 1`.
     I.e. it is "blurred" to the first two loop rounds.
   - **2 bit randomness**: `random_loop_order = [0..3]`: <br/>
     In this case the lookup of the first key bit is blurred to the first 4 loop rounds. 
   - ...
   - **5 bit randomness**: `random_loop_order = [0..31]`: <br/>
     The highest security level can be achieved when blurring the lookups randomly across the 32 loop rounds.
     One specific lookup is then randomly distributed across among all others.

## Naming conventions

All captured traces follow a certain naming scheme corresponding to the activated security countermeasures:

```
<SOURCE>_<SECURITY>_<VARIATION>
```

where 

- `<SOURCE>` describes the source of the trace; e.g. `cwarm` for ChipWhisperer Lite ARM or `elmohwpower` for ELMO HammingWeight Power Model.
- `<SECURITY>` describes the activated countermeasures. E.g. `loop1` enabled 1 bit randomness in the order of the loop execution, whereas `loop5` blurs one lookup across all $2^5 - 1 = 31$ possibilities.
- `<VARIATION>` describes which input part varies: `fixedkey` means the key is fixed while the input contains randomly chosen bytes which are randomly chosen for each trace.